## 加载微调框架PEFT和STF监督微调训练器STFTrainer

In [ ]:
# %%capture #不显示安装过程
import sys, torch
import pandas as pd 
print(sys.executable)
print(torch.__version__, torch.version.cuda)
!pip install huggingface-hub


!pip install unsloth
# Also get the latest nightly Unsloth!
# !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install --upgrade unsloth

In [2]:
# from unsloth import FastLanguageModel
# import torch
# max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
# dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# # 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
#     "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
#     "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
#     "unsloth/llama-3-8b-Instruct-bnb-4bit",
#     "unsloth/llama-3-70b-bnb-4bit",
#     "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
#     "unsloth/Phi-3-medium-4k-instruct",
#     "unsloth/mistral-7b-bnb-4bit",
#     "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
# ] # More models at https://huggingface.co/unsloth

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
#     # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
# )

In [3]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 本地模型目录路径
local_model_path = "/code/ysh/finetuning/data/models/llama-3-8b-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=local_model_path,  # <-- 使用本地路径
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
print("模型和分词器已从本地路径成功加载。")

[HAMI-core Msg(28280:140096873600832:libvgpu.c:852)]: Initialized


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


[HAMI-core Msg(28390:140251720230720:libvgpu.c:836)]: Initializing.....
/usr/local/lib/python3.12/dist-packages/transformers/loss/loss_for_object_detection.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.2)
  from scipy.optimize import linear_sum_assignment
[HAMI-core Msg(28390:140251720230720:libvgpu.c:852)]: Initialized


Unsloth: OpenAI failed to import - ignoring for now.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.55.0.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.25 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


[HAMI-core Msg(28280:140096873600832:memory.c:512)]: orig free=24872288256 total=25393692672 limit=24964497408 usage=402653184
[HAMI-core Msg(28280:140096873600832:memory.c:512)]: orig free=24872288256 total=25393692672 limit=24964497408 usage=402653184


模型和分词器已从本地路径成功加载。


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.8.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


##   从这里开始准备数据集

In [5]:
from datasets import load_dataset
dataset = load_dataset("parquet",data_files='./data/finetuning_data_train.parquet', split = "train")
print(dataset.column_names)

Generating train split: 0 examples [00:00, ? examples/s]

['instruction', 'input', 'output', 'text']


In [6]:
# # df = pd.read_parquet('./data/alpaca-gpt4/train.parquet')
# df = pd.read_parquet('data/finetuning_data_train.parquet')

In [7]:
##必须只有两列——一个 instruction 列和一个 output 列。

In [8]:
## 构造assistant单轮对话模式
from unsloth import to_sharegpt
dataset = to_sharegpt(
    dataset,
    merged_prompt = "{instruction}[[\nYour input is:\n{input}]]",
    output_column_name = "output",
    conversation_extension = 1, # Select more to handle longer conversations
)

Merging columns:   0%|          | 0/4832 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/4832 [00:00<?, ? examples/s]

In [9]:
dataset[0]

{'conversations': [{'from': 'human',
   'value': "Filter exactly 10 unique item IDs that the user may be interested in from the given CandidateItems, based solely on the user's personal profile and rated movie preferences. Only output the item IDs, one per line, with no explanations, no extra text, and no duplicates.based on the user's personal profile. UserID: 1, UserProfile: UserID:1 | Gender:F | Age:1 | Occupation:10 | Zip:48067 | RatedMovies:MovieID:527,rating:5,Genres:Drama|War;MovieID:1545,rating:4,Genres:Drama;MovieID:2398,rating:4,Genres:Drama;MovieID:594,rating:4,Genres:Animation|Children's|Musical;MovieID:1197,rating:3,Genres:Action|Adventure|Comedy|Romance;MovieID:1029,rating:5,Genres:Animation|Children's|Musical;MovieID:2321,rating:3,Genres:Comedy;MovieID:2791,rating:4,Genres:Comedy;MovieID:3114,rating:4,Genres:Animation|Children's|Comedy;MovieID:531,rating:4,Genres:Children's|Drama;MovieID:2918,rating:4,Genres:Comedy;MovieID:661,rating:3,Genres:Animation|Children's|Musical

In [10]:
##将数据集转换为 ShareGPT 格式 的标准化函数
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
"""
{
  "conversations": [
    {
      "from": "human",
      "value": "用户的问题或指令"
    },
    {
      "from": "gpt", 
      "value": "模型的回答"
    }
  ]
}

==>

{
  "conversations": [
    {
      "role": "user",       # 标准化角色名
      "content": "内容..."   # 标准化字段名
    },
    {
      "role": "assistant",  # 标准化角色名
      "content": "内容..."   # 标准化字段名
    }
  ]
}
"""

Unsloth: Standardizing formats (num_proc=96):   0%|          | 0/4832 [00:00<?, ? examples/s]

'\n{\n  "conversations": [\n    {\n      "from": "human",\n      "value": "用户的问题或指令"\n    },\n    {\n      "from": "gpt", \n      "value": "模型的回答"\n    }\n  ]\n}\n\n==>\n\n{\n  "conversations": [\n    {\n      "role": "user",       # 标准化角色名\n      "content": "内容..."   # 标准化字段名\n    },\n    {\n      "role": "assistant",  # 标准化角色名\n      "content": "内容..."   # 标准化字段名\n    }\n  ]\n}\n'

In [11]:
dataset[5]

{'conversations': [{'content': "Filter exactly 10 unique item IDs that the user may be interested in from the given CandidateItems, based solely on the user's personal profile and rated movie preferences. Only output the item IDs, one per line, with no explanations, no extra text, and no duplicates.based on the user's personal profile. UserID: 7, UserProfile: UserID:7 | Gender:M | Age:35 | Occupation:1 | Zip:06810 | RatedMovies:MovieID:3256,rating:5,Genres:Action|Thriller;MovieID:474,rating:5,Genres:Action|Thriller;MovieID:6,rating:4,Genres:Action|Crime|Thriller;MovieID:733,rating:5,Genres:Action|Adventure|Thriller;MovieID:1580,rating:4,Genres:Action|Adventure|Comedy|Sci-Fi;MovieID:2916,rating:5,Genres:Action|Adventure|Sci-Fi|Thriller;MovieID:377,rating:3,Genres:Action|Romance|Thriller;MovieID:3418,rating:3,Genres:Action|Drama;MovieID:589,rating:5,Genres:Action|Sci-Fi|Thriller;MovieID:457,rating:5,Genres:Action|Thriller;MovieID:2571,rating:5,Genres:Action|Sci-Fi|Thriller;MovieID:110,ra

In [12]:
##prompt模版

In [13]:
##Alpaca 格式
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [14]:
## Llama-3 的提示格式，必须使用 instruct 而不是 base 
chat_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{SYSTEM}<|eot_id|><|start_header_id|>user<|end_header_id|>

{INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{OUTPUT}<|eot_id|>"""

In [15]:
##ChatML 格式
chat_template = """<|im_start|>system
{SYSTEM}<|im_end|>
<|im_start|>user
{INPUT}<|im_end|>
<|im_start|>assistant
{OUTPUT}<|im_end|>"""

In [16]:
##最终使用openai模版，一问一答格式
chat_template = """Strictly follow the following requirements to complete the recommendation of candidate items.
### Instruction:
{INPUT}

### Response:
{OUTPUT}"""

from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/4832 [00:00<?, ? examples/s]

In [17]:
dataset[5]

{'conversations': [{'content': "Filter exactly 10 unique item IDs that the user may be interested in from the given CandidateItems, based solely on the user's personal profile and rated movie preferences. Only output the item IDs, one per line, with no explanations, no extra text, and no duplicates.based on the user's personal profile. UserID: 7, UserProfile: UserID:7 | Gender:M | Age:35 | Occupation:1 | Zip:06810 | RatedMovies:MovieID:3256,rating:5,Genres:Action|Thriller;MovieID:474,rating:5,Genres:Action|Thriller;MovieID:6,rating:4,Genres:Action|Crime|Thriller;MovieID:733,rating:5,Genres:Action|Adventure|Thriller;MovieID:1580,rating:4,Genres:Action|Adventure|Comedy|Sci-Fi;MovieID:2916,rating:5,Genres:Action|Adventure|Sci-Fi|Thriller;MovieID:377,rating:3,Genres:Action|Romance|Thriller;MovieID:3418,rating:3,Genres:Action|Drama;MovieID:589,rating:5,Genres:Action|Sci-Fi|Thriller;MovieID:457,rating:5,Genres:Action|Thriller;MovieID:2571,rating:5,Genres:Action|Sci-Fi|Thriller;MovieID:110,ra

In [18]:
# 查看第一个样本
print("Sample 1:")
print(dataset[0]['text'])
print("\n" + "="*50 + "\n")

# # 查看第二个样本
# print("Sample 2:")
# print(dataset[1]['text'])

Sample 1:
<|begin_of_text|>Strictly follow the following requirements to complete the recommendation of candidate items.
### Instruction:
Filter exactly 10 unique item IDs that the user may be interested in from the given CandidateItems, based solely on the user's personal profile and rated movie preferences. Only output the item IDs, one per line, with no explanations, no extra text, and no duplicates.based on the user's personal profile. UserID: 1, UserProfile: UserID:1 | Gender:F | Age:1 | Occupation:10 | Zip:48067 | RatedMovies:MovieID:527,rating:5,Genres:Drama|War;MovieID:1545,rating:4,Genres:Drama;MovieID:2398,rating:4,Genres:Drama;MovieID:594,rating:4,Genres:Animation|Children's|Musical;MovieID:1197,rating:3,Genres:Action|Adventure|Comedy|Romance;MovieID:1029,rating:5,Genres:Animation|Children's|Musical;MovieID:2321,rating:3,Genres:Comedy;MovieID:2791,rating:4,Genres:Comedy;MovieID:3114,rating:4,Genres:Animation|Children's|Comedy;MovieID:531,rating:4,Genres:Children's|Drama;Movi

In [19]:
###Huggingface TRL 的 SFTTrainer
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        # num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

[HAMI-core Msg(29083:139897613728640:multiprocess_memory_limit.c:500)]: Calling exit handler 29083
[HAMI-core Msg(29084:140562999098240:multiprocess_memory_limit.c:500)]: Calling exit handler 29084


Unsloth: Tokenizing ["text"]:   0%|          | 0/4832 [00:00<?, ? examples/s]

In [20]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.25 GB.
7.117 GB of memory reserved.


In [21]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,832 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.262500
2,1.352200
3,1.354800
4,1.257000
5,1.130800
6,1.181000
7,1.221200
8,0.955200
9,1.083200
10,1.025200


In [22]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

213.5678 seconds used for training.
3.56 minutes used for training.
Peak reserved memory = 7.117 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 30.611 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [23]:
## 使用微调结果进行推理 UserID: 195
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {
        "role": "user",
        "content":"Filter exactly 10 unique item IDs that the user may be interested in from the given CandidateItems, based solely on the user's personal profile.Only output the item IDs, one per line, with no explanations, no extra text, and no duplicates.based on the user's personal profile. UserID: 195, UserProfile: UserID:195 | Gender:M | Age:25 | Occupation:12 | Zip:10458 | RatedMovies:MovieID:1228,rating:4,Genres:Drama;MovieID:1587,rating:3,Genres:Action|Adventure;MovieID:141,rating:3,Genres:Comedy;MovieID:3785,rating:2,Genres:Comedy|Horror;MovieID:111,rating:5,Genres:Drama|Thriller;MovieID:2059,rating:3,Genres:Children's|Drama;MovieID:1207,rating:5,Genres:Drama;MovieID:2826,rating:4,Genres:Action|Horror|Thriller;MovieID:36,rating:4,Genres:Drama;MovieID:3451,rating:4,Genres:Comedy|Drama;MovieID:1732,rating:4,Genres:Comedy|Crime|Mystery|Thriller;MovieID:2662,rating:4,Genres:Action|Sci-Fi|War;MovieID:1952,rating:5,Genres:Drama;MovieID:2028,rating:4,Genres:Action|Drama|War;MovieID:1635,rating:4,Genres:Drama;MovieID:709,rating:3,Genres:Animation|Children's;MovieID:67,rating:4,Genres:Drama;MovieID:3822,rating:5,Genres:Drama|Romance;MovieID:3510,rating:4,Genres:Drama|Thriller;MovieID:2125,rating:4,Genres:Drama|Romance;MovieID:2844,rating:4,Genres:Drama|Mystery;MovieID:2968,rating:5,Genres:Adventure|Fantasy|Sci-Fi;MovieID:1729,rating:5,Genres:Crime|Drama;MovieID:3273,rating:2,Genres:Horror|Mystery|Thriller;MovieID:3911,rating:5,Genres:Comedy;MovieID:1655,rating:3,Genres:Horror;MovieID:2804,rating:5,Genres:Comedy|Drama;MovieID:1203,rating:5,Genres:Drama;MovieID:3158,rating:4,Genres:Drama;MovieID:2483,rating:4,Genres:Comedy|Horror|Thriller;MovieID:233,rating:4,Genres:Drama;MovieID:3835,rating:1,Genres:Thriller;MovieID:2770,rating:3,Genres:Comedy;MovieID:1518,rating:2,Genres:Action|Thriller;MovieID:85,rating:4,Genres:Drama|Romance;MovieID:2124,rating:3,Genres:Comedy;MovieID:1274,rating:4,Genres:Adventure|Animation|Sci-Fi|Thriller;MovieID:969,rating:3,Genres:Action|Adventure|Romance|War;MovieID:1253,rating:5,Genres:Drama|Sci-Fi;MovieID:495,rating:5,Genres:Drama;MovieID:3867,rating:5,Genres:Drama;MovieID:1294,rating:4,Genres:Comedy|War;MovieID:2683,rating:3,Genres:Comedy;MovieID:1517,rating:2,Genres:Comedy;MovieID:1944,rating:5,Genres:Drama|Romance|War;MovieID:3266,rating:5,Genres:Action|Comedy|Crime|Drama;MovieID:774,rating:4,Genres:Drama;MovieID:2097,rating:3,Genres:Children's|Horror;MovieID:2762,rating:4,Genres:Thriller;MovieID:2615,rating:3,Genres:Adventure|Sci-Fi;MovieID:2134,rating:2,Genres:Comedy;MovieID:3787,rating:5,Genres:Comedy;MovieID:300,rating:5,Genres:Drama;MovieID:866,rating:4,Genres:Crime|Drama|Romance|Thriller;MovieID:3054,rating:4,Genres:Animation|Children's;MovieID:2793,rating:3,Genres:Comedy|Horror;MovieID:1390,rating:3,Genres:Comedy;MovieID:3481,rating:4,Genres:Comedy;MovieID:919,rating:5,Genres:Adventure|Children's|Drama|Musical;MovieID:858,rating:5,Genres:Action|Crime|Drama;MovieID:2689,rating:3,Genres:Documentary;MovieID:899,rating:5,Genres:Musical|Romance;MovieID:2654,rating:4,Genres:Horror;MovieID:223,rating:3,Genres:Comedy;MovieID:2607,rating:4,Genres:Drama;MovieID:3502,rating:5,Genres:Drama;MovieID:426,rating:2,Genres:Horror|Sci-Fi|Thriller;MovieID:435,rating:3,Genres:Comedy|Sci-Fi;MovieID:1215,rating:4,Genres:Action|Adventure|Comedy|Horror|Sci-Fi;MovieID:1221,rating:5,Genres:Action|Crime|Drama;MovieID:1261,rating:4,Genres:Action|Adventure|Comedy|Horror;MovieID:1676,rating:5,Genres:Action|Adventure|Sci-Fi|War;MovieID:3358,rating:3,Genres:Comedy|Romance;MovieID:3608,rating:3,Genres:Comedy;MovieID:3704,rating:3,Genres:Action|Sci-Fi;MovieID:1867,rating:3,Genres:Action|Adventure;MovieID:778,rating:4,Genres:Drama;MovieID:810,rating:2,Genres:Children's|Comedy|Fantasy;MovieID:1321,rating:4,Genres:Horror;MovieID:928,rating:5,Genres:Romance|Thriller;MovieID:1204,rating:5,Genres:Adventure|War;MovieID:2273,rating:3,Genres:Action|Thriller;MovieID:3624,rating:4,Genres:Action;MovieID:1591,rating:4,Genres:Action|Adventure|Sci-Fi|Thriller;MovieID:1845,rating:5,Genres:Comedy|Thriller;MovieID:1909,rating:4,Genres:Mystery|Sci-Fi|Thriller;MovieID:2359,rating:4,Genres:Comedy;MovieID:1779,rating:2,Genres:Adventure|Sci-Fi|Thriller;MovieID:1862,rating:2,Genres:Horror|Sci-Fi;MovieID:1754,rating:2,Genres:Action|Mystery|Thriller;MovieID:1911,rating:3,Genres:Comedy;MovieID:2026,rating:3,Genres:Horror|Thriller;MovieID:1132,rating:5,Genres:Drama;MovieID:1731,rating:3,Genres:Comedy;MovieID:1161,rating:4,Genres:Drama;MovieID:3156,rating:3,Genres:Comedy|Drama|Sci-Fi;MovieID:3753,rating:4,Genres:Action|Drama|War;MovieID:3355,rating:3,Genres:Thriller;MovieID:2391,rating:4,Genres:Crime|Thriller;MovieID:851,rating:3,Genres:Drama,\n CandidateItems: 3944:Comedy|Drama;3563:Action|Horror;2517:Horror;2607:Drama;551:Children's|Comedy|Musical;3764:Action|Crime|Thriller;2219:Mystery|Thriller;1356:Action|Adventure|Sci-Fi;688:Action|Adventure|Comedy|War;1702:Children's|Comedy|Fantasy;2127:Drama|Romance;1677:Comedy;2347:Action;237:Comedy|Romance;1047:Action|Thriller;3093:Drama|Western;2660:Sci-Fi;3392:Comedy;2206:Mystery|Thriller;3740:Action|Comedy;2064:Comedy|Documentary;2545:Comedy;3721:Comedy;2644:Horror;3778:Comedy|Drama;2247:Comedy;2807:Action|Sci-Fi;2632:Drama;1301:Sci-Fi;3156:Comedy|Drama|Sci-Fi;464:Action|Adventure|Crime|Thriller;329:Action|Adventure|Sci-Fi;2253:Action|Comedy|Fantasy;2490:Action|Thriller;1556:Action|Romance|Thriller;821:Romance;2788:Comedy;634:Comedy;3007:Documentary;506:Drama;1871:Comedy|Drama;1375:Action|Adventure|Sci-Fi;1183:Drama|Romance|War;2339:Comedy|Romance;1473:Action|Comedy|Crime|Drama;2831:Drama;1374:Action|Adventure|Sci-Fi;2770:Comedy;169:Adventure|Children's|Drama;3436:Drama|Romance.\n"
    },
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(
    input_ids, 
    streamer = text_streamer, 
    max_new_tokens = 128, 
    pad_token_id = tokenizer.eos_token_id,
    temperature=0.01,  # ← 添加温度控制
    do_sample=True,   # ← 必须开启采样才能让 temperature 起作用！
)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Recommended movie ID: [3764, 329, 2339, 1375, 1183, 506, 1374, 2770, 169, 3436]<|end_of_text|>


In [24]:
###UserID: 195 'Recommended movie ID: [3007, 1183, 1702, 3764, 1301, 2247, 1356, 329, 1374, 1375]'

## 保存、加载微调模型

In [25]:
# 这仅保存了 LoRA 适配器，并未保存整个模型。要保存为 16 位或 GGUF，请用下面的
model.save_pretrained("outputs/lora_model") # Local saving
tokenizer.save_pretrained("outputs/lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('outputs/lora_model/tokenizer_config.json',
 'outputs/lora_model/special_tokens_map.json',
 'outputs/lora_model/chat_template.jinja',
 'outputs/lora_model/tokenizer.json')

In [26]:
##加载训练好的lora块进行推理

In [ ]:
##加载模型，使用微调结果进行推理 UID：618
##如果想加载刚刚保存的 LoRA 适配器以进行推理，请将 False 更改为 True
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "outputs/lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
pass

messages = [                    
    # Change below!
    {"role": "user", "content": "Filter exactly 10 unique item IDs that the user may be interested in from the given CandidateItems, based solely on the user's personal profile and rated movie preferences. Only output the item IDs, one per line, with no explanations, no extra text, and no duplicates.based on the user's personal profile. UserID: 618, UserProfile: UserID:618 | Gender:M | Age:25 | Occupation:0 | Zip:74105 | RatedMovies:MovieID:1970,rating:3,Genres:Horror;MovieID:1971,rating:3,Genres:Horror;MovieID:3596,rating:1,Genres:Comedy;MovieID:490,rating:2,Genres:Thriller;MovieID:2792,rating:2,Genres:Comedy;MovieID:3755,rating:3,Genres:Action|Adventure|Thriller;MovieID:648,rating:2,Genres:Action|Adventure|Mystery;MovieID:3409,rating:3,Genres:Drama|Thriller;MovieID:3513,rating:3,Genres:Drama|Thriller;MovieID:70,rating:4,Genres:Action|Comedy|Crime|Horror|Thriller;MovieID:2454,rating:3,Genres:Horror|Sci-Fi;MovieID:3947,rating:3,Genres:Thriller;MovieID:3918,rating:2,Genres:Horror;MovieID:1965,rating:2,Genres:Comedy|Sci-Fi;MovieID:3917,rating:4,Genres:Horror;MovieID:3930,rating:2,Genres:Horror;MovieID:1171,rating:4,Genres:Comedy;MovieID:3624,rating:4,Genres:Action;MovieID:2291,rating:3,Genres:Drama|Romance;MovieID:2791,rating:4,Genres:Comedy;MovieID:1673,rating:3,Genres:Drama;MovieID:3481,rating:4,Genres:Comedy;MovieID:551,rating:2,Genres:Children's|Comedy|Musical;MovieID:3578,rating:4,Genres:Action|Drama;MovieID:589,rating:4,Genres:Action|Sci-Fi|Thriller;MovieID:1249,rating:2,Genres:Thriller;MovieID:2858,rating:5,Genres:Comedy|Drama;MovieID:1259,rating:4,Genres:Adventure|Comedy|Drama;MovieID:1250,rating:5,Genres:Drama|War;MovieID:110,rating:4,Genres:Action|Drama|War;MovieID:3347,rating:4,Genres:Drama;MovieID:2717,rating:3,Genres:Comedy|Horror;MovieID:1196,rating:4,Genres:Action|Adventure|Drama|Sci-Fi|War;MovieID:1130,rating:2,Genres:Horror;MovieID:2144,rating:3,Genres:Comedy;MovieID:2746,rating:3,Genres:Comedy|Horror|Musical,\n CandidateItems: 507:Action|Drama;3428:Drama;3579:Drama;1222:Action|Drama|War;1969:Horror;280:Drama|Thriller;2084:Children's|Musical;3628:Action|Drama|War;1982:Horror;2160:Horror|Thriller;1544:Action|Adventure|Sci-Fi|Thriller;1126:Comedy|Fantasy;2624:Drama;2142:Animation|Children's|Comedy;1216:Adventure|Romance;1230:Comedy|Romance;1952:Drama;1036:Action|Thriller;2991:Action;1829:Drama|Romance;3148:Drama;769:Documentary;1357:Drama|Romance;1977:Horror;1551:Adventure|Children's|Drama;1486:Drama;3160:Drama;2249:Comedy;1405:Animation|Comedy;3291:Thriller;2012:Comedy|Sci-Fi|Western;3048:Comedy;1504:Drama;1976:Horror;2881:Action|Thriller;2456:Horror|Sci-Fi;1972:Horror;3534:Comedy;434:Action|Adventure|Crime;935:Comedy|Musical;2407:Comedy|Sci-Fi;2996:Drama;2708:Romance;864:Comedy|Drama;3902:Drama;1254:Adventure;611:Action|Horror|Sci-Fi;3490:Horror;1986:Horror.\n"
    },
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
# _ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

##控温
_ = model.generate(
    input_ids,
    streamer=text_streamer,
    max_new_tokens=128,
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.01,  # ← 添加温度控制
    do_sample=True,   # ← 必须开启采样才能让 temperature 起作用！
)

==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.55.0.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.25 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


[HAMI-core Msg(28280:140096873600832:memory.c:512)]: orig free=17572102144 total=25393692672 limit=24964497408 usage=419694592
[HAMI-core Msg(28280:140096873600832:memory.c:512)]: orig free=17572102144 total=25393692672 limit=24964497408 usage=419694592


In [ ]:
## 618'Recommended movie ID: [2456, 1976, 1969, 3579, 1972, 1977, 1986, 611, 2624]'

In [ ]:
## uid 541 
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "outputs/lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
pass
messages = [                    
    {"role": "user", 
     "content":"Filter exactly 10 unique item IDs that the user may be interested in from the given CandidateItems, based solely on the user's personal profile and rated movie preferences. Only output the item IDs, one per line, with no explanations, no extra text, and no duplicates.based on the user's personal profile. UserID: 541, UserProfile: UserID:541 | Gender:F | Age:18 | Occupation:4 | Zip:5849574 | RatedMovies:MovieID:1276,rating:5,Genres:Comedy|Drama;MovieID:1225,rating:5,Genres:Drama;MovieID:608,rating:4,Genres:Crime|Drama|Thriller;MovieID:1197,rating:3,Genres:Action|Adventure|Comedy|Romance;MovieID:923,rating:2,Genres:Drama;MovieID:1198,rating:5,Genres:Action|Adventure;MovieID:318,rating:5,Genres:Drama;MovieID:1204,rating:2,Genres:Adventure|War;MovieID:1207,rating:2,Genres:Drama;MovieID:3469,rating:2,Genres:Drama;MovieID:3440,rating:1,Genres:Action|Children's|Fantasy;MovieID:3439,rating:1,Genres:Action|Children's|Fantasy;MovieID:60,rating:3,Genres:Adventure|Children's|Fantasy;MovieID:3489,rating:5,Genres:Adventure|Fantasy;MovieID:2,rating:2,Genres:Adventure|Children's|Fantasy;MovieID:3438,rating:5,Genres:Action|Children's|Fantasy;MovieID:653,rating:3,Genres:Action|Adventure|Fantasy;MovieID:2622,rating:2,Genres:Comedy|Fantasy;MovieID:367,rating:3,Genres:Comedy|Crime|Fantasy;MovieID:788,rating:3,Genres:Comedy|Fantasy|Romance|Sci-Fi;MovieID:317,rating:3,Genres:Children's|Comedy|Fantasy;MovieID:3247,rating:3,Genres:Comedy|Crime;MovieID:356,rating:5,Genres:Comedy|Romance|War;MovieID:3255,rating:2,Genres:Comedy|Drama;MovieID:1500,rating:5,Genres:Comedy|Crime;MovieID:1923,rating:5,Genres:Comedy;MovieID:1784,rating:5,Genres:Comedy|Drama;MovieID:141,rating:3,Genres:Comedy;MovieID:11,rating:3,Genres:Comedy|Drama|Romance;MovieID:3253,rating:4,Genres:Comedy;MovieID:235,rating:4,Genres:Comedy|Drama;MovieID:342,rating:2,Genres:Comedy|Romance;MovieID:2997,rating:5,Genres:Comedy;MovieID:21,rating:4,Genres:Action|Comedy|Drama;MovieID:2321,rating:3,Genres:Comedy;MovieID:39,rating:4,Genres:Comedy|Romance;MovieID:1641,rating:5,Genres:Comedy;MovieID:497,rating:3,Genres:Comedy|Romance;MovieID:223,rating:3,Genres:Comedy;MovieID:1588,rating:5,Genres:Children's|Comedy;MovieID:1265,rating:4,Genres:Comedy|Romance;MovieID:1136,rating:5,Genres:Comedy;MovieID:34,rating:5,Genres:Children's|Comedy|Drama;MovieID:2396,rating:5,Genres:Comedy|Romance;MovieID:2858,rating:5,Genres:Comedy|Drama;MovieID:880,rating:1,Genres:Sci-Fi|Thriller;MovieID:435,rating:1,Genres:Comedy|Sci-Fi;MovieID:2053,rating:1,Genres:Children's|Comedy|Sci-Fi;MovieID:546,rating:1,Genres:Action|Adventure|Children's|Sci-Fi;MovieID:512,rating:1,Genres:Horror|Sci-Fi;MovieID:1690,rating:1,Genres:Action|Horror|Sci-Fi;MovieID:2094,rating:1,Genres:Action|Adventure|Sci-Fi;MovieID:327,rating:1,Genres:Action|Comedy|Musical|Sci-Fi;MovieID:1676,rating:1,Genres:Action|Adventure|Sci-Fi|War;MovieID:3269,rating:1,Genres:Adventure|Romance|Sci-Fi;MovieID:1320,rating:2,Genres:Action|Horror|Sci-Fi|Thriller;MovieID:185,rating:3,Genres:Sci-Fi|Thriller;MovieID:196,rating:1,Genres:Horror|Sci-Fi;MovieID:2012,rating:2,Genres:Comedy|Sci-Fi|Western;MovieID:2105,rating:2,Genres:Action|Adventure|Fantasy|Sci-Fi;MovieID:426,rating:1,Genres:Horror|Sci-Fi|Thriller;MovieID:3660,rating:1,Genres:Horror|Sci-Fi|Thriller;MovieID:2011,rating:3,Genres:Comedy|Sci-Fi;MovieID:329,rating:3,Genres:Action|Adventure|Sci-Fi;MovieID:2046,rating:2,Genres:Adventure|Children's|Sci-Fi;MovieID:2021,rating:1,Genres:Fantasy|Sci-Fi;MovieID:2699,rating:1,Genres:Action|Comedy|Sci-Fi|Thriller;MovieID:2054,rating:4,Genres:Adventure|Children's|Comedy|Fantasy|Sci-Fi;MovieID:2600,rating:1,Genres:Action|Sci-Fi|Thriller;MovieID:1527,rating:1,Genres:Action|Sci-Fi;MovieID:1396,rating:2,Genres:Crime|Drama|Sci-Fi;MovieID:2140,rating:3,Genres:Children's|Fantasy|Sci-Fi;MovieID:780,rating:2,Genres:Action|Sci-Fi|War;MovieID:2985,rating:2,Genres:Action|Crime|Sci-Fi;MovieID:1653,rating:3,Genres:Drama|Sci-Fi|Thriller;MovieID:1580,rating:5,Genres:Action|Adventure|Comedy|Sci-Fi;MovieID:3175,rating:5,Genres:Adventure|Comedy|Sci-Fi;MovieID:2657,rating:3,Genres:Comedy|Horror|Musical|Sci-Fi;MovieID:1356,rating:5,Genres:Action|Adventure|Sci-Fi;MovieID:3702,rating:2,Genres:Action|Sci-Fi;MovieID:2916,rating:4,Genres:Action|Adventure|Sci-Fi|Thriller;MovieID:1584,rating:4,Genres:Drama|Sci-Fi;MovieID:1127,rating:2,Genres:Action|Adventure|Sci-Fi|Thriller;MovieID:1210,rating:4,Genres:Action|Adventure|Romance|Sci-Fi|War;MovieID:480,rating:5,Genres:Action|Adventure|Sci-Fi;MovieID:32,rating:4,Genres:Drama|Sci-Fi;MovieID:2529,rating:3,Genres:Action|Sci-Fi;MovieID:589,rating:4,Genres:Action|Sci-Fi|Thriller;MovieID:1270,rating:3,Genres:Comedy|Sci-Fi;MovieID:541,rating:3,Genres:Film-Noir|Sci-Fi;MovieID:1200,rating:4,Genres:Action|Sci-Fi|Thriller|War;MovieID:1214,rating:3,Genres:Action|Horror|Sci-Fi|Thriller;MovieID:1196,rating:4,Genres:Action|Adventure|Drama|Sci-Fi|War;MovieID:2092,rating:1,Genres:Animation|Children's|Musical;MovieID:239,rating:2,Genres:Animation|Children's|Comedy|Romance;MovieID:631,rating:1,Genres:Animation|Children's|Musical;MovieID:13,rating:2,Genres:Animation|Children's;MovieID:2123,rating:3,Genres:Animation|Children's;MovieID:1064,rating:3,Genres:Animation|Children's|Comedy;MovieID:1030,rating:3,Genres:Adventure|Animation|Children's|Musical,\n CandidateItems: 3732:Horror;508:Drama;1632:Comedy|Romance;1645:Crime|Horror|Mystery|Thriller;1587:Action|Adventure;1887:Adventure|Comedy;2170:Action|Comedy;3796:Romance|Thriller;1464:Mystery;145:Action;239:Animation|Children's|Comedy|Romance;24:Drama|Sci-Fi;2678:Drama;1854:Comedy|Romance;2520:Drama;1423:Drama;763:Drama;926:Drama;193:Drama;1670:Drama|War;538:Drama;1889:Thriller;809:Action|Adventure;838:Comedy|Drama|Romance;2632:Drama;3936:Drama|Thriller;458:Drama|Western;587:Comedy|Romance|Thriller;3725:Animation|Musical;2633:Horror|Romance;935:Comedy|Musical;3418:Action|Drama;590:Adventure|Drama|Western;1404:Crime|Drama;2532:Action|Sci-Fi;1121:Drama;1222:Action|Drama|War;3174:Comedy|Drama;261:Drama;3443:Action|Drama|Thriller;3495:Comedy|Drama;2857:Animation|Musical;3367:War;2112:Crime|Drama;1639:Drama|Romance;1370:Action|Thriller;1730:Drama;62:Drama;2502:Comedy|Romance;1485:Comedy.\n"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
# _ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)
_ = model.generate(
    input_ids,
    streamer=text_streamer,
    max_new_tokens=128,
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.01 ,  # ← 添加温度控制
    do_sample=True,   # ← 必须开启采样才能让 temperature 起作用！
)

In [ ]:
## 541 :'Recommended movie ID: [508, 3418, 62, 3174, 838, 587, 1639, 1370, 1485, 590]'